In [ ]:
!nvidia-smi
import os
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/My Drive'
os.chdir(path)
os.listdir(path)

In [ ]:
#Epoch: One forward pass and one backward pass of all the training examples(所有数据完成一次训练)
#Batch-Size: The number of training examples in one forward backward pass.(每次参数更新使用的样本数量，一个Epoch常分割为几个Batch)
#Iteration: Number of passes, each pass using [batch size] number of examples. (一个Epoch迭代的次数，即Batch的个数。10000条数据，Batch-size为1000的话，Iteration为10)
#shuffle=True 增加随机性打乱顺序

import torch
from torch.utils.data import Dataset #Dateset是抽象类(abstract class)，无法直接实例化，只可创建子类继承后，实例化子类
from torch.utils.data import DataLoader #加载数据，shuffle打乱数据,分Batch
import numpy as np


class DiabetesDataset(Dataset):
  def __init__(self, filepath):
    xy = np.loadtxt(filepath, delimiter=',' , dtype=np.float32, skiprows = 1)
    '''
    loadtxt(fname,                  #要读取的文件、文件名、或生成器。
            dtype=<class 'float'>,  #数据类型，默认float。
            comments='#',           #注释
            delimiter=None,         #分隔符，默认是空格。
            converters=None, 
            skiprows=0,             #跳过前几行读取，默认是0，必须是int整型。
            usecols=None,           #要读取哪些列，0是第一列。例如，usecols = （1,4,5）将提取第2，第5和第6列。默认读取所有列。
            unpack=False,           #如果为True，将分列读取。
            ndmin=0)
    '''
    self.len = xy.shape[0]#取出数据集的第一个纬度，即有几行数据，数据的条数。这里数据集的纬度为(n*9)，第一个纬度为n
    self.x_data = torch.from_numpy(xy[:, :-1])#取所有行，除最后一列外的所有列
    self.y_data = torch.from_numpy(xy[:, [-1]])#取所有行，最后一列
  
  #支持下标index操作 dataset[index]
  def __getitem__(self,index):
    return self.x_data[index],self.y_data[index]
  
  #返回数据集内的数据条数
  def __len__(self):
    return self.len


class Model(torch.nn.Module):
  def __init__(self):
    super(Model,self).__init__()
    self.linear1 = torch.nn.Linear(8,6)
    self.linear2 = torch.nn.Linear(6,4)
    self.linear3 = torch.nn.Linear(4,1)
    #构建一个3层网络，神经元数量分别为8,6,4。整体上为输入一个8纬特征，输出一个二分类结果
    
    self.activate = torch.nn.ReLU()
    #创建激活函数模块，直接可在下方forward用self调用
    self.sigmoid = torch.nn.Sigmoid()

  def forward(self, x):
    x = self.activate(self.linear1(x))
    x = self.activate(self.linear2(x))
    x = self.sigmoid(self.linear3(x))
    #最后一层输出，为计算loss，由ReLU改为sigmoid
    return x


dataset = DiabetesDataset('./Colab Notebooks/dataset/diabetes.csv')
train_loader = DataLoader(dataset = dataset,#传递数据集
                          batch_size = 32,  
                          shuffle = True,
                          num_workers = 2)  #读取数据时，并行的进行数量

model = Model()
criterion = torch.nn.BCELoss(size_average=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)


loss_list = []
epoch = 10
for i in range(epoch):
  for j,data in enumerate(train_loader):
    inputs,labels = data 
    y_pred = model(inputs)
    loss = criterion(y_pred,labels)

    loss_list.append(loss)
    #print(i,loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    #print('----------------------------------------------')


plt.plot(range(epoch*24),loss_list) #768条数据，batch_size32,一个epoch分为24组
plt.xlabel('batch')
plt.ylabel('loss')
plt.show()
